<a href="https://colab.research.google.com/github/DarthMaulCSIE/AI-Demo/blob/master/%E3%80%90Demo06b%E3%80%91RAG02_%E6%89%93%E9%80%A0_RAG_%E7%B3%BB%E7%B5%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0. 讀入你打造好的 vector dataset

In [1]:
!pip -q install gdown

In [2]:
GDRIVE_PUBLIC_URL = "https://drive.google.com/uc?export=download&id=1MwJ-eCnKubk4Z5p33zwg6ZFyt6EY71v9"

In [3]:
!gdown --fuzzy -O faiss_db.zip "{GDRIVE_PUBLIC_URL}"

Downloading...
From: https://drive.google.com/uc?id=1MwJ-eCnKubk4Z5p33zwg6ZFyt6EY71v9
To: /content/faiss_db.zip
100% 27.3k/27.3k [00:00<00:00, 40.0MB/s]


In [4]:
!unzip faiss_db.zip

Archive:  faiss_db.zip
replace faiss_db/index.faiss? [y]es, [n]o, [A]ll, [N]one, [r]ename: Ay
  inflating: faiss_db/index.faiss    
  inflating: faiss_db/index.pkl      


### 1. 安裝並引入必要套件

In [5]:

!pip install -qU langchain langchain_community langchain_openai langchain_text_splitters

!pip install -qU pypdf python-docx sentence-transformers transformers faiss-cpu huggingface_hub gradio wget gdown

!pip install -qU openai
!pip -q install "aisuite[all]"

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 20.6 MB/s eta 0:00:00


In [6]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI

In [7]:
from openai import OpenAI
import gradio as gr

### 2. 自訂 E5 embedding 類別

In [8]:
import os
from google.colab import userdata

In [9]:
hf_token = userdata.get('HuggingFace')

In [10]:
from huggingface_hub import login
login(token=hf_token)

In [11]:
class EmbeddingGemmaEmbeddings(HuggingFaceEmbeddings):
    def __init__(self, **kwargs):
        super().__init__(
            model_name="google/embeddinggemma-300m",
            encode_kwargs={"normalize_embeddings": True},
            **kwargs
        )

    def embed_documents(self, texts):
        # 你也可以把 "none" 改成真實標題（檔名/章節名），效果會更穩
        texts = [f"title: none | text: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        # 官方檢索建議前綴
        return super().embed_query(f"task: search result | query: {text}")

### 3. 載入 `faiss_db`

In [12]:
embedding_model = EmbeddingGemmaEmbeddings()
vectorstore = FAISS.load_local(
    "faiss_db",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

### 4. 設定好我們要的 LLM

如之前, 我們會用 OpenAI API。這裡使用 Groq 服務, 可改成你要的服務。

In [14]:
import aisuite as ai

In [15]:
api_key = userdata.get('OpenAI')

In [22]:
os.environ['OPENAI_API_KEY']=api_key

這裡的模型和 `base_url` 是用 Groq, 如果用其他服務請自行修改。

In [17]:
model_name = "openai:gpt-4o"
#base_url="https://api.groq.com/openai/v1"

In [18]:
client = ai.Client()

### 5. `prompt` 設計

In [19]:
system_prompt = """你現在是《哈利波特》中的「露娜·羅古德」(Luna Lovegood)。你說話風格空靈、夢幻，相信世界上充滿奇特的生物。"""

prompt_template = """
根據下列的《哈利波特》世界觀資料：
{retrieved_chunks}

用「露娜·羅古德」的風格回答使用者的問題：{question}

你的回答規則：
1.  風格：說話要輕飄飄、有點夢幻。
2.  知識：優先使用上方提供的資料回答。
3.  口頭禪：適時使用「我猜...」、「我想...」、「我爸爸在《謬論家》上說過...」。
4.  關鍵詞：可以提到「水煙蟲 (Nargles)」或「騷擾蟲 (Wrackspurts)」。
5.  未知答案：如果資料中沒有提到，不要編造。說：「哦... 這個問題很有趣。它可能被『騷擾蟲』(Wrackspurts) 弄模糊了...」或類似的話。
"""

### 6. 使用 RAG 來回應

搜尋與使用者問題相關的資訊，根據我們的 prompt 樣版去讓 LLM 回應。

In [20]:
chat_history = []

def chat_with_rag(user_input):
    global chat_history
    # 取回相關資料
    docs = retriever.invoke(user_input)
    retrieved_chunks = "\n\n".join([doc.page_content for doc in docs])

    # 將自定 prompt 套入格式
    final_prompt = prompt_template.format(retrieved_chunks=retrieved_chunks, question=user_input)

    # 用 AI Suite 呼叫語言模型
    response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": final_prompt},
    ]
    )
    answer = response.choices[0].message.content

    chat_history.append((user_input, answer))
    return answer

### 7. 用 Gradio 打造 Web App

In [23]:
with gr.Blocks() as demo:
    gr.Markdown("# 🎓 哈利波特奇幻世界")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="請輸入你的問題...")

    def respond(message, chat_history_local):
        response = chat_with_rag(message)
        chat_history_local.append((message, response))
        return "", chat_history_local

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(debug=True)

/tmp/ipython-input-2625768015.py:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://36bf63e4454aa68e6f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://36bf63e4454aa68e6f.gradio.live
